# New buildings with housing

In [23]:
import polars as pl
import pandas as pd

In [24]:
file_path = 'new_building.xlsx'

df_pandas = pd.read_excel(file_path)

In [25]:
annees = df_pandas.iloc[1, 3:].astype(int).tolist()
print(f"Années disponibles : {annees}")

df_pandas = df_pandas.iloc[2:].reset_index(drop=True)

colonnes = ['code_commune', 'nom_commune', 'type_batiment'] + [str(annee) for annee in annees]
df_pandas.columns = colonnes

print(f"\nNombre total de lignes : {len(df_pandas)}")

Années disponibles : [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

Nombre total de lignes : 2193


In [26]:
df_pandas = df_pandas[df_pandas['type_batiment'] == 'Bâtiments avec logements - Total'].copy()

print(f"Nombre de communes : {len(df_pandas)}")
print(f"\nAperçu des types de bâtiments uniques :")
print(df_pandas['type_batiment'].unique())

Nombre de communes : 2161

Aperçu des types de bâtiments uniques :
['Bâtiments avec logements - Total']


In [27]:
# keep commune + years
colonnes_a_garder = ['nom_commune'] + [str(annee) for annee in annees]
df_pandas = df_pandas[colonnes_a_garder]

print("Colonnes conservées :")
print(df_pandas.columns.tolist())

Colonnes conservées :
['nom_commune', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']


In [28]:
# Convert in Polars dataframe
df_polars = pl.from_pandas(df_pandas)
df_polars = df_polars.with_columns(
    pl.col("nom_commune").str.replace(r"^\.+\d+ ", "").alias("nom_commune")
)
# Only commune
df_polars = df_polars.filter(
    ~pl.col("nom_commune").str.starts_with("-") & 
    ~pl.col("nom_commune").str.starts_with(">>") 
)

print("✓ DataFrame Polars créé avec succès!")
print(f"Dimensions: {df_polars.shape}")
df_polars.head(10)

✓ DataFrame Polars créé avec succès!
Dimensions: (2136, 12)


nom_commune,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Aeugst am Albis""",0.0,1.0,0.0,2.0,5.0,11.0,5.0,3.0,1.0,4.0,1.0
"""Affoltern am Albis""",14.0,9.0,10.0,8.0,11.0,9.0,11.0,9.0,7.0,8.0,8.0
"""Bonstetten""",6.0,6.0,2.0,5.0,12.0,1.0,5.0,1.0,3.0,2.0,2.0
"""Hausen am Albis""",14.0,3.0,5.0,30.0,12.0,3.0,2.0,3.0,7.0,8.0,2.0
"""Hedingen""",6.0,10.0,4.0,0.0,3.0,1.0,3.0,2.0,2.0,8.0,6.0
"""Kappel am Albis""",3.0,13.0,1.0,14.0,2.0,12.0,2.0,6.0,4.0,3.0,0.0
"""Knonau""",2.0,6.0,7.0,42.0,5.0,4.0,0.0,3.0,3.0,3.0,2.0
"""Maschwanden""",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Mettmenstetten""",4.0,11.0,12.0,52.0,5.0,19.0,5.0,17.0,26.0,6.0,21.0


# trend score

In [29]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [30]:
df_long = df_polars.melt(
    id_vars=['nom_commune'],
    variable_name='annee',
    value_name='nombre_logements'
)

df_long = df_long.with_columns(
    pl.col('annee').cast(pl.Int32)
)

print(f"Dataset : {df_long.shape[0]} lignes, {len(df_long['nom_commune'].unique())} communes")
df_long.head()

Dataset : 23496 lignes, 2136 communes


/tmp/ipykernel_22037/2804221204.py:1: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_long = df_polars.melt(


nom_commune,annee,nombre_logements
str,i32,f64
"""Aeugst am Albis""",2013,0.0
"""Affoltern am Albis""",2013,14.0
"""Bonstetten""",2013,6.0
"""Hausen am Albis""",2013,14.0
"""Hedingen""",2013,6.0


In [31]:
def calculate_trend_features(df_long):
    """
    Calculer les features de tendance pour chaque commune
    basé uniquement sur les données de logements 2013-2023
    """
    
    features_list = []
    
    for commune in df_long['nom_commune'].unique():
        data = df_long.filter(pl.col('nom_commune') == commune).sort('annee')
        
        annees = data['annee'].to_numpy()
        logements = data['nombre_logements'].to_numpy()
        
        # 1. Pente de la tendance linéaire (régression)
        if len(annees) > 1:
            slope = np.polyfit(annees, logements, 1)[0]
        else:
            slope = 0
        
        # 2. Croissance totale (2023 vs 2013)
        if len(logements) >= 2:
            croissance_totale = logements[-1] - logements[0]
            taux_croissance = ((logements[-1] - logements[0]) / (logements[0] + 1)) * 100
        else:
            croissance_totale = 0
            taux_croissance = 0
        
        # 3. Moyenne des 3 dernières années vs 3 premières années
        if len(logements) >= 6:
            recent_avg = np.mean(logements[-3:])
            ancien_avg = np.mean(logements[:3])
            acceleration = recent_avg - ancien_avg
        else:
            acceleration = 0
        
        # 4. Total cumulé (volume global)
        total_logements = np.sum(logements)
        
        # 5. Moyenne annuelle
        avg_logements = np.mean(logements)
        
        # 6. Volatilité (écart-type)
        volatilite = np.std(logements)
        
        # 7. Tendance récente (pente sur les 5 dernières années)
        if len(annees) >= 5:
            recent_slope = np.polyfit(annees[-5:], logements[-5:], 1)[0]
        else:
            recent_slope = slope
        
        # 8. Maximum atteint
        max_logements = np.max(logements)
        
        features_list.append({
            'nom_commune': commune,
            'slope': slope,
            'croissance_totale': croissance_totale,
            'taux_croissance': taux_croissance,
            'acceleration': acceleration,
            'total_logements': total_logements,
            'avg_logements': avg_logements,
            'volatilite': volatilite,
            'recent_slope': recent_slope,
            'max_logements': max_logements
        })
    
    return pl.DataFrame(features_list)

# Calculer les features
df_features = calculate_trend_features(df_long)
print(f"\nFeatures calculées pour {df_features.shape[0]} communes")
df_features.head(10)


Features calculées pour 2136 communes


nom_commune,slope,croissance_totale,taux_croissance,acceleration,total_logements,avg_logements,volatilite,recent_slope,max_logements
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Penthalaz""",-0.318182,0.0,0.0,-3.333333,16.0,1.454545,1.924183,-0.3,6.0
"""Oberhünigen""",0.009091,0.0,0.0,0.0,1.0,0.090909,0.28748,-0.2,1.0
"""Schlieren""",-0.818182,-8.0,-57.142857,-9.0,94.0,8.545455,5.758214,-1.4,20.0
"""Flüelen""",-0.163636,-2.0,-66.666667,-2.0,23.0,2.090909,1.378705,-0.2,5.0
"""Dagmersellen""",-0.281818,-2.0,-15.384615,-3.0,146.0,13.272727,2.299838,-0.6,18.0
"""Büetigen""",-0.027273,1.0,100.0,-1.0,36.0,3.272727,2.092884,-0.8,8.0
"""Laufenburg""",-0.463636,-1.0,-14.285714,-3.0,65.0,5.909091,3.579094,-0.1,11.0
"""Crans-Montana""",-1.118182,-7.0,-21.212121,-6.333333,270.0,24.545455,6.787202,1.5,35.0
"""Les Breuleux""",-0.227273,-6.0,-66.666667,-2.0,49.0,4.454545,2.104698,-0.1,8.0


In [32]:
def modele_3_acceleration(df_features):
    """
    Modèle focalisé sur l'accélération de la croissance
    Bon pour détecter les "futures stars"
    
    Ce modèle privilégie :
    - L'accélération de la croissance (40%)
    - Le momentum (différence entre tendance récente et passée) (30%)
    - La tendance récente (20%)
    - Le taux de croissance global (10%)
    """
    df = df_features.to_pandas()
    scaler = MinMaxScaler(feature_range=(0, 100))
    
    df_scaled = df.copy()
    
    # Normaliser les features
    features_to_scale = ['slope', 'acceleration', 'recent_slope', 
                         'taux_croissance', 'max_logements']
    
    for feature in features_to_scale:
        df_scaled[f'{feature}_scaled'] = scaler.fit_transform(df[[feature]])
    
    # Calculer le momentum = différence entre recent_slope et slope général
    # Un momentum positif signifie que la croissance s'accélère
    df_scaled['momentum'] = df_scaled['recent_slope_scaled'] - df_scaled['slope_scaled']
    df_scaled['momentum_scaled'] = scaler.fit_transform(df_scaled[['momentum']])
    
    # Calculer le trend score avec des poids orientés accélération
    df_scaled['trend_score'] = (
        0.40 * df_scaled['acceleration_scaled'] +     # Accélération (40%)
        0.30 * df_scaled['momentum_scaled'] +         # Momentum (30%)
        0.20 * df_scaled['recent_slope_scaled'] +     # Tendance récente (20%)
        0.10 * df_scaled['taux_croissance_scaled']    # Taux de croissance (10%)
    )
    
    result = df_scaled[['nom_commune', 'trend_score']].copy()
    return pl.from_pandas(result).sort('trend_score', descending=True)

# Calculer les scores avec le modèle 3
df_trend_scores = modele_3_acceleration(df_features)

print("\n=== MODÈLE 3 : POTENTIEL D'ACCÉLÉRATION ===")
print("Ce modèle identifie les communes dont la croissance s'accélère\n")
print(f"Trend scores calculés pour {df_trend_scores.shape[0]} communes")
df_trend_scores.head(20)


=== MODÈLE 3 : POTENTIEL D'ACCÉLÉRATION ===
Ce modèle identifie les communes dont la croissance s'accélère

Trend scores calculés pour 2136 communes


nom_commune,trend_score
str,f64
"""Zürich""",69.670583
"""Troinex""",69.629474
"""Niederweningen""",66.556983
"""Birrhard""",64.997286
"""Renens (VD)""",64.862413
…,…
"""Alpnach""",62.381795
"""Bodio""",62.373273
"""Echallens""",62.075156


In [33]:
# Sauvegarder en CSV
output_file = 'building_trend.csv'
df_trend_scores.write_csv(output_file)
print(f"✅ Fichier '{output_file}' créé avec succès!")


# Afficher le résultat final
df_trend_scores

✅ Fichier 'building_trend.csv' créé avec succès!


nom_commune,trend_score
str,f64
"""Zürich""",69.670583
"""Troinex""",69.629474
"""Niederweningen""",66.556983
"""Birrhard""",64.997286
"""Renens (VD)""",64.862413
…,…
"""Val de Bagnes""",38.486714
"""Bellinzona""",35.045685
"""Bussigny""",34.725588
